# Project Description

## Analyzing Impact of V2I connectivity in terms of traffic

In this project the main objective is to understand the impact of vehicle-to-infrastructure (V2I) communication in traffic systems. In particular the focus will be oriented towards the creation of different kind of messages and understanding the effects of those messages in the traffic network.

<img src="assets/img/01_v2i.jpg" alt="drawing" width="600"/>


In order to do that we are going to make use of the package `connectv2x` that includes some basic models and communication for `V2I` systems. For more information about the source code of this package please go [here](https://github.com/research-licit/connectv2x). Over there you will find a folder called `connectv2x` containing all the models in the submodules `carfollow`, `vehicles`, `demand`, `messages` etc. Some of these will be explored along the module and may help you to progress faster in the project.   

In [ ]:
# Try this on google collab only
# !pip install -r https://raw.githubusercontent.com/research-licit/ITSProjects/main/requirements.txt

## Contents and structure 

* Exploration and modeling of vehicle dynamics
* Traffic indicators 
* Analyzing different scenarios of V2I messaging 
* Compute emissions from trajectories 
* Scaling up and analyzing deep characteristics. 

Let's begin by importing some plotting libraries that will help us with data visualization

In [ ]:
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

## Understanding the basics 

For this application it is considered the simulation of [microscopic traffic models](https://en.wikipedia.org/wiki/Microscopic_traffic_flow_model) where *longitudinal position* follow a specified behavior defined as a function of  two main components, the *headway space* and the *speed differential*. The *car following* behavior describes the behavior of the vehicle in its longitudinal dynamics while the *lane change* behavior describes the behavior in the lateral position. 

In order to modify traffic behavior for a condition, the system is modeled via traffic model where V2I messages modify vehicle speed or lateral position   

#### Traffic model and the fundamental diagram

Before pursuing it is important to understand the value of the fundamental diagram. The fundamental diagram describe the relation ship between ***density*** or concentration of vehicles and the ***flow*** or speed of vehicles. The density is regularly denoted as $\kappa$ for this scenarios as `K`, in the meanwhile the flow is denoted by `Q`. We are going to consider the fact that the relationship follows a piecewise linear behavior as follows:


\begin{aligned}
Q(\kappa) = \begin{cases}
u\kappa\quad 0\leq\kappa\leq \kappa_c\\
w\left(\kappa_x-\kappa\right)\quad \kappa_c\leq\kappa\leq \kappa_x
\end{cases}
\end{aligned}


Let's define the number of vehicles and their initial positions, in order to determine the initial position let's find the minimum spacing. For the moment let's consider the following parameters. `W=5`, `K_X=0.2` and `U=20`. Let's trace the fundamental diagram for this case. We will make use of the `FundamentalDiagram` class in the `traffic` submodule

In [ ]:
from itstools.connectv2x.traffic import FundamentalDiagram
from bokeh.plotting import show

f = FundamentalDiagram(w=5,u=25,k_x=0.2)
p = f.plot_diagram()
show(p)

We can easily establish the ***critical density*** which corresponds to the amount of vehicles at which there is maximum flow via

In [ ]:
print(f"Capacity: {f.C}")
print(f"Critical density: {f.k_c}")
print(f"Maximum density: {f.k_x}")
print(f"Speed limit: {f.u}")
print(f"Congestion speed: {f.w}")

The critical density represent the minimum tolerable inter-vehicle distance **critical headway space** before vehicles start to decrease their speed. On the other hand when the value of density is maximum, and the flow is 0, the vehicles respect the **minimum headway space** that can be computed from the maximum value of density $\kappa_x$ via

\begin{aligned}
\kappa_x = \frac{N}{d} \approx \frac{1}{s_0}  \approx \frac{1}{l_{\text{avg}}+s_{\text{min}}} 
\end{aligned}

Where $l_{\text{avg}}$ is the average vehicle length and $s_{text{min}}$ represents the minimum tolerated headway spacing between two vehicles. 

By computing the case in the fundamental diagram

In [ ]:
print(f"Minimum headway space: {1/f.k_x}")

**Q1**: Supose a vehicle has an average length of $l_{\text{avg}}=4$ [m] and the minimum tolerable inter vehicle distance is $s_{\text{min}}=2.25$[m]. Consider the congestion wave speed $w=6.25$ [m/s] and a free flow speed of $u=25$ [m/s]. Compute and plot the fundamental diagram for this case and find the value of the capacity and minimum headway space:

<span style= "color:blue">Provide your answers below:</span>

#### Car following behavior and traffic model 

Keep in mind the constants `K_X`, `W`, `U` you have computed, they will be useful for building up the behavior of cars in traffic. 

For the sake of clarity, the following corresponds to the notations for variable description in the model. It is considered the vehicle position of a vehicle as $x_n$ and the headway space between a vehicle and its leader as $s_n = x_{n-1}-x_{n}$. The vehicle's speed and acceleration are defined as $v_n$,$a_n$ respectively. The operator $\Delta v_n = v_{n-1} - v_{n}$ refers to the difference between the leading vehicle and the following one. 

For a determined vehicle in the network the longitudinal dynamics are determined by the acceleration behavior. In this case it is considred Tampere's Law. 

$$ 
a_n(t+T_n) = \min \left(c_{1,n-1}\Delta v_{n-1,n} + 
c_{2}\left(\Delta x_{n,n-1} - \left(s_0+\tau v_n(t)\right)\right),
c_{3}\left(v^\star(t) - v_n(t)\right)\right)
$$

One of the main features of this model is the adaptability to a specific speed condition, while preserving properties of the traffic such as the car following behavior in congestion situation. This feature makes it possible to trace features in the fundamental diagram. 

*To implement the model a `class` object called `Tampere` has been implemented. The class intends to describe the full behavior of the vehicle.* 

#### Parameters 

So far parameters in the model have been fixed although random scenarios can be also considered.

| Parameter     | Value     | Units |
:--------------:|:---------:|:------:
$$c_1,c_2,c_3$$ | 0.5       |
$$\tau$$        | $$\frac{1}{wk_x}$$ | [s]
$$w$$           | $$6.25$$  | [m/s]
$$k_x$$         | $$0.16$$  | [veh/km]
$$u_i$$         | $$25$$    | [m/s]

Please follow step by step variable definitions for more detail into simulations. Let define the parameters declared in the table as `C_1,C_2,C_3,W,K_X,W` and import the class `Tampere` from the submodule `connectv2x.carfollow`. 

In [ ]:
from itstools.connectv2x.carfollow import Tampere

C_1, C_2, C_3 = (0.5,0.5,0.5)

In the following we are going to compute the initial speeds `V_0`, positions `X_0` for simulating the vehicle's dynamic. Let's consider for the moment a set of two vehicles 

In [ ]:
import numpy as np 

# Initialize the vehicles. Run from here to repeat the dynamic evolution 

N_VEH = 2 # Number of vehicles 
X0 = np.array([25,0])
V0 = np.ones(N_VEH) * f.u
A0 = np.zeros(N_VEH)
V_CLASS = ["HDV", "HDV"]

veh_list = []
Tampere.reset()
for x0, v0, vtype in zip(X0, V0, V_CLASS):
    veh_list.append(Tampere(x0=x0, v0=v0, veh_type=vtype))
print(f"List of vehicles: {veh_list}")

Up to the moment vehicles have been created only, it is required to link them in order to indicate who is the leader and who is the follower. For that it is possible to use the method `set_leader`

In [ ]:
veh_list[1].set_leader(veh_list[0])
print(f"Leader for vehicle {veh_list[0].idx}: {veh_list[0].veh_lead}")
print(f"Leader for vehicle {veh_list[1].idx}: {veh_list[1].veh_lead.idx}")

Based on the definition of the fundamental diagram let's compute the parameters required to evolve the acceleration equation. In general in order to evolve the acceleration equation it is required to solve the following dynamical system: 


\begin{aligned}
x_n(t+T) &= x_n(t) + v_n(t)T\\
v_n(t+T) &= v_n(t) + a_n(t) T\\
\end{aligned}

In order to evolve this set of equations in time it is required to account for the initial positions for all vehicles and in particular for a particular definition of $a_0(t)$ the first vehicle in the formation.

#### Setting a desired control speed 

Let's consider the *sigmoid* function as a principle to create a change in speed via the equation : 

$$
S(A,a,d,B,x) = \frac{A}{\textstyle 1+e^{\displaystyle-\frac{\left(x-d\right)}{a}}}+B
$$

In [ ]:
# Sigmoid function 
from itstools.connectv2x.support import sigmoid
from bokeh.plotting import figure 
from bokeh.layouts import column 
from bokeh.palettes import Spectral5

x = np.linspace(-6,6,100)

A_range = np.linspace(1,2,5)
d_range = np.linspace(0,2,5)
a_range = np.linspace(0.5,2,5)
B_range = np.linspace(0,1,5)

# Figure creation 
p = [figure(title="S(t)",plot_height=350,plot_width=500) for _ in range(4)]

for A,d,a,b, color in zip(A_range,d_range,a_range, B_range, Spectral5):
    p[0].line(x,sigmoid(x,A,1,0),color=color,legend_label = f"A: {A}")
    p[1].line(x,sigmoid(x,1,1,d),color=color,legend_label = f"d: {d}")
    p[2].line(x,sigmoid(x,1,a,0),color=color,legend_label = f"a: {a}")
    p[3].line(x,sigmoid(x,1,1,0)+b,color=color,legend_label = f"B: {b}")

    
p[0].legend.location = "top_left";
p[1].legend.location = "top_left"
p[2].legend.location = "top_left"
p[3].legend.location = "top_left"

show(column(p[0],p[1],p[2],p[3]))

Now you have seen how to transform the signal such that it can be *amplified*, *delayed*, *compressed*, *displaced* according to the parameters `A`, `d`, `a`,`B` respectively 

**Q2:** Consider the sigmoid function presented before. Create a `distance` vector of $1000$[m] with steps of 1 meter and create a sigmoid function that starts a signal at $25$ [m/s] and drops down the speed to $10$[m/s] in an interval of $300$[m]. Start this maneuver at around $200$[m]

<span style= "color:blue">Provide your answers below:</span>

##### Main dynamic evolution 

In order to evolve the system it is necessary a loop going step by step and computing the position of each vehicle. In this case we consider the scenario under which the leader vehicle is affected by a profile of speed. In this case we consider the following case: 

In [ ]:
# Road works speed profile
from itstools.connectv2x.support import speed_pulse 

def lead_spd(x):
    """  Leader's function to control speed drop in space 
         Speed Drop: 20 m/s 
         Position: 15 Km
         Duration: 20 Km
    """
    return speed_pulse(x, drop=20, delay=5000, duration=2000)

x_t = np.linspace(0, 10000, 10000)
v_t = lead_spd(x_t)

p = figure(title="V(x)",plot_height=350,plot_width=500)
p.line(x_t, v_t, legend_label="Lead speed")
p.legend.location = "bottom_left"
p.xaxis.axis_label = "Position [m]"
p.yaxis.axis_label = "Speed [m/s]"
show(p)

#### Time iteration 

In this case we consider the application of the former profile. As you can see there is a function `lead_spd` that computes the speed as a function of the distance `x`. It is possible to create multiple functions broadcasting messages in this case. 

In [ ]:
# Dynamical evolution 
def evolve_dynamics(veh_list, lead_spd, X0,V0,A0):
    X = X0
    V = V0
    A = A0

    # Declaring time vector
    T_STEP = 960
    time = np.linspace(0,T_STEP,T_STEP)

    for t in time:
        for veh in veh_list:
            veh.step_evolution(control=lead_spd)

        V = np.vstack((V, np.array([veh.v for veh in veh_list])))
        X = np.vstack((X, np.array([veh.x for veh in veh_list])))
        A = np.vstack((A, np.array([veh.a for veh in veh_list])))

    V = V[1:, :]
    X = X[1:, :]
    A = A[1:, :]
    return time, X,V,A 

time, X,V,A = evolve_dynamics(veh_list,lead_spd, X0,V0,A0)

In [ ]:
from itstools.connectv2x.plottools import plot_xva

zooms = (
(0,10000),
(-1,26),
(-5,5)
)

titles = (
    f"X-Time ",
    f"V-Time ",
    f"X-Time "
  )
pos, spd, acc = plot_xva(time, X, V, A, y_range=zooms, titles=titles)
show(column(pos,spd,acc))

**Q3:** Determine and find based on the data given before find the `headway space` defined as $x_{n-1}-x_n$. Plot the variation of the headway space in as a function of `time` and as a function of `X`. 

*Note*: `X`, `V`,`A` are matrices where the columns represent information  particular for a vehicle and the rows is the variable at a specific time instant e.g `X[10,0]` is the position of the first vehicle at time instant `10` 

<span style= "color:blue">Provide your answers below:</span>

**Q4:** Based on the function created in the step *Q2* create a function that receives as an input the position `x` and returns the speed of the leading vehicle. 

<span style= "color:blue">Provide your answers below:</span>

In [ ]:
# Prototype a function as 

def my_leader_speed(x):
    # Compute the leader speed as a function of the position. 
    return v

**Q5:** Based on the function `evolve_dynamics` given by : 

```{python}
# Dynamical evolution 
X = X0
V = V0
A = A0

for t in time:
    for veh in veh_list:
        veh.step_evolution(control=lead_spd)

    V = np.vstack((V, np.array([veh.v for veh in veh_list])))
    X = np.vstack((X, np.array([veh.x for veh in veh_list])))
    A = np.vstack((A, np.array([veh.a for veh in veh_list])))
```

Determine the evolution of the vehicles and trace the curves `Position`, `Speed` and `Acceleration` as a function of time. 

<span style= "color:blue">Provide your answers below:</span>

#### Scaling up the number of vehicles 

We have seen up the moment a simulation with very few vehicles. We are interested in observing the same behavior with more vehicles. Let's consider the case where vehicles are now around `20`. In this case we well consider the case where the vehicles are spaced at double of the minimum allowed vehicle distance $x_{n-1}-x_{n} = 3 * s_x$ 

In [ ]:
# Initialize the network. Run from here to repeat the dynamic evolution 

def initialize_network():
    N_VEH = 20 # Number of vehicles 
    X0 = np.flip(np.arange(0, N_VEH) * f.s_x * 4)
    V0 = np.ones(N_VEH) * f.u
    A0 = np.zeros(N_VEH)
    V_CLASS = ["HDV" for _ in range(N_VEH)]

    veh_list = []

    # Initializing vehicles
    Tampere.reset()
    for x0, v0, vtype in zip(X0, V0, V_CLASS):
        veh_list.append(Tampere(x0=x0, v0=v0, l0=0, veh_type=vtype))

    # Setting leader for vehicle i
    for i in range(1, N_VEH):
        veh_list[i].set_leader(veh_list[i - 1])

    return veh_list,X0,V0,A0

veh_list,X0,V0,A0 = initialize_network()

In [ ]:
# Evolution of the system 
time, X,V,A = evolve_dynamics(veh_list, lead_spd, X0,V0,A0)

pos, spd, acc = plot_xva(time, X, V, A, y_range=zooms, titles=titles)
show(column(pos,spd,acc))

**Q6**: Create a list of $50$ vehicles. Set the initial condition of the vehicle such that vehicles are spaced at an average distance of $3*s_x$. Determine this value from the fundamental diagram. Set the initial condition of all vehicles in $25$ [m/s] and set the speed of the leader as the function created in the step *Q4* `my_lead_speed`. Compute the evolution of all vehicles and store the answers in the variables `Xa`,`Va`,`Aa` 

<span style= "color:blue">Provide your answers below:</span>

#### Compute Total Travel Time 

To compute the total travel time we consider the time taken from the position where the leader starts its trip until a specified distance in kilometers `d`. We make use of `pandas` to filter data, the module `datetime` to find the time diference and the method `apply` to compute the values for all vehicles in the network. Examine the code down below for details on this computation. 

In [ ]:
import datetime  as dt 
import pandas as pd 

# Creation of Pandas dataframe 
df_x = pd.DataFrame(X)

now = dt.datetime(2019,12,17,13,0) # This is to set a special day for synchronous purposes 
delta = dt.timedelta(seconds=1)
time_vector = [now+n*delta for n in range(int(np.max(time)))]
df_x.index = time_vector 

# Compute the total travel time from 0 to Km 8000
def find_travel_time(df,max_dist = 8000):
    time_entry  = min(df[df>df_x.iloc[0,0]].index) # minimum time after leader entrance 
    time_exit = max(df[df<max_dist].index) # maximum time before exit
    travel_time = time_exit-time_entry
    return travel_time.total_seconds()

ttt = df_x.apply(find_travel_time)
tt_mean,tt_var = np.mean(ttt),np.std(ttt) 
ttt = sum(ttt)

print(f"Mean Travel time: {tt_mean}, Variance Travel time: {tt_var}, Total Traveltime:{ttt}")

#### Broadcasting messages from infrastructure towards vehicles 

Up to the moment all vehicle classes have been denominated `HDV` standing for human driven vehicle. Let's say we want to send a message to a vehicle in the formation, we are then interested in broadcasting messages for vehicles, in order to do that we are going to make use of a special vehicle class denominated `CAV` (connected and automated vehicle), those vehicles will correspond to connected vehicles and they will be perceiving messages from the infrastructure. Let's reinitialize the network of vehicles. 

In [ ]:
# Initialize the network. Run from here to repeat the dynamic evolution 
veh_list,X0,V0,A0 = initialize_network()

Let suppose that the vehicle with index $10$ is going to be a connected one, then

In [ ]:
# Modify a single vehicle type: 
veh_list[10].type = "CAV"

print([veh.type for veh in veh_list])

The objective of the message sent to the `CAV` will be to indicate a decrease in speed. Due to the presence of a congestion. We expect to send this message starting from $2000$[m] to the connected vehicles. 

In [ ]:
from itstools.connectv2x.messages import Msg2

send_message = Msg2
d_accept = 2000
tx_message = []

for veh in veh_list:
    tx_message.append(send_message(d_accept))
    
x_ss = np.linspace(0,8000,10000)
    
acc_values = np.array(list(map(lambda x: x(x_ss), tx_message)))

p = figure(title="Transmitted message to CAV")
p.line(x_ss,acc_values[0])
p.xaxis.axis_label = "Position [m]"
p.yaxis.axis_label = "Speed [m/s]"
show(p)

Let's compute the dynamical evolution of the new system with the function `evolve_dynamics_cav`. Note that in general the function has the same behavior as `evolve_dynamics` it only changes by registering the message to the `CAV` 

In [ ]:
# Dynamical evalution
def evolve_dynamics_cav(veh_list, lead_spd, X0,V0,A0):
    X = X0
    V = V0
    A = A0

    send_message = Msg2  # Msg2 # Defines the type of message to be send

    d_accept = 2000 # Distance at which the message is transmitted 

    for t in time:
        for veh in veh_list:
            if veh.type == "CAV" and not veh.acc:
                msg = send_message(d_accept)
                veh.register_control_speed(msg)

            veh.step_evolution(control=lead_spd)

        V = np.vstack((V, np.array([veh.v for veh in veh_list])))
        X = np.vstack((X, np.array([veh.x for veh in veh_list])))
        A = np.vstack((A, np.array([veh.a for veh in veh_list])))

    V = V[1:, :]
    X = X[1:, :]
    A = A[1:, :]
    return time, X,V,A 

time, X_CAV,V_CAV,A_CAV = evolve_dynamics_cav(veh_list,lead_spd, X0,V0,A0)    
pos, spd, acc = plot_xva(time, X_CAV, V_CAV, A_CAV, y_range=zooms, titles=titles)
show(column(pos,spd,acc))

**Q7**: For the case of the `CAV` compute again the *total travel time* and plot the *headway space* in terms of time. How has the messages impacted the traffic? 

<span style= "color:blue">Provide your answers below:</span>

**Q8**: Now select the message `my_lead_speed` and a random number of `10` vehicles among the total simulated when obtained `Xa`,`Va`,`Aa`. Apply a similar message like `Msg2` to this case and find the corresponding evolution. Extract the results in matrices `XaCAV`, `VaCAV` and `AaCAV`.

<span style= "color:blue">Provide your answers below:</span>


#### Computing emissions and side effects 

When checking emissions we can refer multiple models, in this case we are going to make use of the model given by [Treiber, 2014](http://traffic-flow-dynamics.org). The function to compute instantaneous $CO_2$ emissions is given by: 

$$\dot{CO}_2 =  2.3 \frac{100000}{C_{\text{spec}}} \frac{P(v,\dot{v})}{v}$$

where 

$$P(v,\dot{v}) = max(P_0+ v F(v,\dot{v}),0)$$

$$F(v,\dot{v}) = m\dot{v} + (\mu+\phi)mg + \frac{1}{2}\rho c_d A v^2$$

Some of these parameters are: 

| Parameter     | Value     | Units |
:--------------:|:---------:|:------:
$$m$$           | 1500      | [kg]
$$P_0$$         | 3         | [kW]
$$C_{\text{spec}}$$| 300    | [ml/kWh]
$$\rho$$        | 1.3       | [kg/m$^3$]
$$\mu$$         | 0.02      | [veh/km]
$$c_d$$         | 0.3       | 
$$A$$           | 2         | [m$^2$]

$\dot{v}$ denote the vehicle acceleration and $v$ denote the vehicle speed

**Q9**: Design a function that accepts the matrices `V` and `A` and returns the emissions of each vehicle in time. Compute the cumulated emissions in time for all vehicles and compare to the case of `V_CAV`,  `A_CAV`. 

<span style= "color:blue">Provide your answers below:</span>


## Project 02

* One of the main concerns when speaking about `V2V` technologies is related to the messages sent from the infrastructure towards the vehicle in this case we have considered a jump in speed from $25$[m/s] to $18$[m/s]. Study the case where the jump is less abrupt e.g $25-22$ and extract the trajectories and speeds. For the case where there are only two connected vehicles. 
* Consider applying this messages to up to 5 vehicles out of 20 in the network at positions $500$[m], $1000$[m], $1500$[m], $2000$[m] and $2500[m]$. Extract the trajectory profiles for this case. Compare two cases when the speed drop have the following characteristics. Situation A: $25$[m/s] to $18$[m/s]. Situation B: $25$[m/s] to $22$[m/s]
* Study the behavior of the average `headway space` when both situations and provide conclusions on how V2I messaging could impact the performance of the network, take into account both situations A and B applied.  
* What is the impact in terms of `total travel time` when the number of vehicles affected by the policy augment?
* Determine the net effect on estimated `CO_2` emissions due to the aplication of both policies. Situation A and Situation B. 
* For which case in terms of distance at which the message is received the behavior of the system performs better?

A. Ladino